In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd
import gym
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
import random
from collections import deque
import sys
import matplotlib.pyplot as plt


In [2]:
class Critic(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Critic, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)

    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)

        return x

class Actor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate = 3e-4):
        super(Actor, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = torch.tanh(self.linear3(x))

        return x



In [3]:
class Noise:
    def __init__(self, action_space_size, mu = 0.0, sigma = 1.0, theta = 0.15):
        self.mu = mu
        self.theta = theta
        self.sigma = sigma
        self.state = None
        self.action_space_size = action_space_size
    
    def reset(self):
        self.state = self.mu * np.ones(self.action_space_size)
    
    def get_state(self):
        x = self.state
        dx = self.theta * (self.mu - self.state) + self.sigma * np.random.rand(self.action_space_size)
        self.state = x + dx
        return self.state
        

In [4]:
class Buffer:
    def __init__(self, size):
        self.memory = deque(maxlen=size)
    
    def add(self, state, action, reward, next_state, done):
        step = (state, action, np.array([reward]), next_state, done)
        self.memory.append(step)
    
    def get_mini_batch(self, size):
        states = []
        actions = []
        rewards = []
        next_states = []
        dones = []
        
        mini_batch = random.sample(self.memory, size)
        for step in mini_batch:
            state, action, reward, next_state, done = step
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            next_states.append(next_state)
            dones.append(done)
            
        return states, actions, rewards, next_states, dones
    
    def get_size(self):
        return len(self.memory)
    

In [5]:
class Agent:
    def __init__(self,states_n, actions_n, hidden_size=256, actor_alpha=1e-4, critic_alpha=1e-3, gamma=0.99, tau=1e-2, max_memory_size=50000):
        self.hidden_size = hidden_size
        self.actor_alpha = actor_alpha
        self.critic_alpha = critic_alpha
        self.gamma = gamma
        self.tau = tau
        self.n_states = states_n
        self.n_actions = actions_n
        #create Actor Network
        
        self.Actor = Actor(self.n_states, self.hidden_size, self.n_actions)
        self.Actor_t = Actor(self.n_states, self.hidden_size, self.n_actions)
        for weight_t, weight in zip(self.Actor_t.parameters(), self.Actor.parameters()):
            weight_t.data.copy_(weight.data)      
        print(self.Actor.parameters()) 
        self.Actor_opt = optim.Adam(self.Actor.parameters(), lr=actor_alpha)
        #create Critic Network
        self.Critic = Critic(self.n_states + self.n_actions, hidden_size, self.n_actions)
        self.Critic_t = Critic(self.n_states + self.n_actions, hidden_size, self.n_actions)
        for weight_t, weight in zip(self.Critic_t.parameters(), self.Critic.parameters()):
            weight_t.data.copy_(weight.data)
        self.Critic_opt = optim.Adam(self.Critic.parameters(), lr=actor_alpha)
        
        #copy weights

        
            
        #create noise
        self.noise = Noise(self.n_actions)
        #create buffer
        self.replay_buffer = Buffer(max_memory_size)
        
    def push_buffer(self, state, action, reward, next_state, done):
        self.replay_buffer.add(state, action, reward, next_state, done)
        #if buffer is full
#         if(self.buffer.get_size() > batch_size):
#             self.update(mini_batch_size)
    
        
    def get_action(self, state):
        state = torch.tensor(state, dtype=torch.float).unsqueeze(0)
        #action = np.clip(action.numpy() + self.noise.get_state(), self.env.action_space.low, self.env.action_space.high)
        action = self.Actor.forward(state)
        action = action.detach().numpy()[0,0] + self.noise.get_state()
        return action
        
    def update(self, mini_batch_size):
        states, actions, rewards, next_states, dones = self.replay_buffer.get_mini_batch(mini_batch_size)
        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        
        #update Critic
        next_actions = self.Actor_t.forward(next_states)
        Q_t_next = self.Critic_t.forward(next_states, next_actions)
        Q_targets = rewards + (self.gamma * Q_t_next)
        Q = self.Critic.forward(states, actions)
        C_loss = F.mse_loss(Q, Q_targets)
        self.Critic_opt.zero_grad()
        C_loss.backward()
        self.Critic_opt.step()
        
        #update Actor
        new_actions = self.Actor.forward(states)
        P_loss = -self.Critic.forward(states, new_actions).mean()
        self.Actor_opt.zero_grad()
        P_loss.backward()
        self.Actor_opt.step()
        
        #update Target network
        for target_param, param in zip(self.Actor_t.parameters(), self.Actor.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
       
        for target_param, param in zip(self.Critic_t.parameters(), self.Critic.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))

    def noise_reset(self):
        self.noise.reset()
        
    def get_buffer(self):
        return self.replay_buffer

In [6]:
env = gym.make('BipedalWalker-v2')
env.reset()

array([ 2.74506630e-03,  1.52753142e-05, -1.99180283e-03, -1.60001028e-02,
        9.26420316e-02,  4.62383591e-03,  8.59622747e-01, -2.27398394e-03,
        1.00000000e+00,  3.29183005e-02,  4.62365011e-03,  8.53454545e-01,
       -3.14610514e-03,  1.00000000e+00,  4.40813214e-01,  4.45819318e-01,
        4.61421937e-01,  4.89549309e-01,  5.34101844e-01,  6.02459908e-01,
        7.09147573e-01,  8.85930181e-01,  1.00000000e+00,  1.00000000e+00])

In [ ]:

batch_size = 128
rewards = []
avg_rewards = []
o = env.observation_space.shape[0]
a = env.action_space.shape[0]
agent = Agent(o, a)

for episode in range(50):
    state = env.reset()
    agent.noise_reset()
    episode_reward = 0
    
    for step in range(500):
        action = agent.get_action(state)
        env.render()
        new_state, reward, done, _ = env.step(action) 
        agent.push_buffer(state, action, reward, new_state, done)
        
        if agent.get_buffer().get_size() > batch_size:
            agent.update(batch_size)        
        
        state = new_state
        episode_reward += reward

        if done:
            sys.stdout.write("episode: {}, reward: {}, average _reward: {} \n".format(episode, np.round(episode_reward, decimals=2), np.mean(rewards[-10:])))
            break

    rewards.append(episode_reward)
    avg_rewards.append(np.mean(rewards[-10:]))

plt.plot(rewards)
plt.plot(avg_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

    

<generator object Module.parameters at 0x10ececf68>


/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


episode: 0, reward: -113.3, average _reward: nan 
episode: 1, reward: -115.12, average _reward: -113.29628317009534 
episode: 2, reward: -115.13, average _reward: -114.20951914203872 
episode: 3, reward: -114.86, average _reward: -114.51476134038826 
episode: 4, reward: -115.8, average _reward: -114.6011160147203 
episode: 5, reward: -119.87, average _reward: -114.84147369422858 
episode: 6, reward: -112.81, average _reward: -115.67962503796696 
episode: 7, reward: -115.97, average _reward: -115.2698633551923 
episode: 8, reward: -115.85, average _reward: -115.35744932156135 


In [ ]:
plt.plot(rewards)
plt.plot(avg_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()